# Sample Code

## 基礎建設

In [2]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [3]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-29 13:32:16--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  42.1MB/s    in 0.4s    

2021-12-29 13:32:16 (42.1 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2021-12-29 13:32:16--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  35.1MB/s    in 0.3s   

In [4]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [5]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [6]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 清理資料

In [7]:
metadata_clean = metadata.drop_duplicates('asin')
print(f'Original size: {len(metadata)}, modify: {len(metadata_clean)}')


Original size: 32892, modify: 32488


In [8]:
# title/ rank string clean
print("rank not str Number:",len(metadata_clean["rank"][metadata_clean['rank'].apply(lambda x: type(x)!=str)]))
print("title not str Number:",len(metadata_clean["title"][metadata_clean['title'].apply(lambda x: type(x)!=str)]))

rank not str Number: 402
title not str Number: 0


In [9]:
metadata_clean["rank"] = metadata_clean["rank"].apply(lambda x: x if type(x) == str else "999999999999 in None")
metadata_clean["title"] = metadata_clean["title"].apply(lambda x: x if type(x) == str else "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 資料整理



1. 將metadata重複asin的清除
2. category目前的項目都是空的
3. 若用rank後的資料分類，有404項為無分類
4. 每個種類的成交數量都明顯下降，感覺公司要倒了...
5. Beauty & Personal Care與None是相對較多購買量的商品種類



In [10]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [11]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [12]:
# ranking setting
metadata_clean.loc[:,'ranking'] = metadata_clean.loc[:,'rank'].str.split(expand = True)[0]
metadata_clean.loc[:,'ranking'] = metadata_clean.loc[:,'ranking'].str.replace(',', '')
metadata_clean.loc[:,'ranking'] = pd.to_numeric(metadata_clean.loc[:,'ranking'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [13]:
# category check
number = len(metadata_clean['category'][metadata_clean['category'].apply(lambda x: len(x)) > 0])
print(f'There are {number} catagory having data.')

There are 0 catagory having data.


In [14]:
metadata_clean["sub_category"] = metadata_clean["rank"].str.split("in ").str[1].replace(r"\(","", regex=True).replace(r"\&amp;","&", regex=True)
metadata_clean["sub_category"].value_counts().rename_axis('name').reset_index(name='counts')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,name,counts
0,Beauty & Personal Care,32015
1,None,402
2,Grocery & Gourmet Food,37
3,Health & Household,21
4,Toys & Games,3
5,Sports & Outdoors,3
6,"Clothing, Shoes & Jewelry",2
7,Baby,2
8,Home & Kitchen,1
9,Automotive,1


### 對sub_category與時間作分析
--> 公司購買各種類的數量明顯下降... (公司危機!!)

In [86]:
metadata_clean[metadata_clean["sub_category"]=="None"].head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,ranking,sub_category,text_data,specify
36,[],,[<P><STRONG>Helps Reduce Appearance Of Hair Gr...,,Nair Hair Remover 4 Minute Lotion with Aloe &a...,[],,nair,[],999999999999 in None,[],"{'Shipping Weight:': '6.4 ounces', 'ASIN: ': '...",All Beauty,,,,B000052WYR,[],[],999999999999,None,nair hair remov 4 minut lotion alo amp lanolin...,28.000
38,[],,[Scope Original Mint Rinse kills bad breath ge...,,Scope Original Mint Mouthwash 50.7 Fl Oz,"[B009LNU6ME, B00L1I1VYY, B00J5K0QEQ, B0035U184...",,Unknown,[],999999999999 in None,"[B009LNU6ME, B007CRIFSS, B074KHNR82, B01FVAYP8...",{' Product Dimensions: ': '5.1 x 3.7 ...,All Beauty,,,,B000052YD8,[],[],999999999999,None,scope origin mint mouthwash 507 fl oz scope or...,28.000
43,[],,[Using this is like chumming the air for prett...,,Old Spice High Endurance Anti-Perspirant &amp;...,[],,Old Spice,[],999999999999 in None,"[B006Y3JDH8, B01I9THZKQ, B07CPFGVM3, B0014CU2U...",{' Product Dimensions: ': '5 x 2.1 x ...,All Beauty,,,$5.21,B00005308B,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,999999999999,None,old spice high endur antiperspir amp deodor or...,29.000
46,[],,[Naturally comfort your skin with this St. Ive...,,St. Ives Advanced Therapy Lotion for Extra Dry...,[],,St. Ives,[],999999999999 in None,[],{' Product Dimensions: ': '8.7 x 3.4 ...,All Beauty,,,,B0000530ND,[],[],999999999999,None,st ive advanc therapi lotion extra dri skin al...,28.000
54,[],,"[, <strong>Platinum Protection is the best dry...",,Secret Platinum Protection Antiperspirant &amp...,[],,Secret,[],999999999999 in None,[],"{'Shipping Weight:': '4.8 ounces', 'Domestic S...",All Beauty,,,,B00005354G,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,999999999999,None,secret platinum protect antiperspir amp deodor...,2.000


In [87]:
def historyCounts(start, end, biastime):
  current = start
  output = []
  index = {name: i for i, name in enumerate(np.unique(metadata_clean["sub_category"]))}
  while current < end:
    tmp = [0]*len(index)
    currentTime = current.strftime("%Y-%m-%d")
    nextTime = (current+biastime).strftime("%Y-%m-%d")
    print(f'{currentTime} -> {nextTime}')
    tmpdata = ratings[(ratings['DATE'] >= currentTime) & (ratings['DATE'] < nextTime)]
    for item, row in tmpdata.iterrows():
      asin = row['asin']
      sub_category = metadata_clean[metadata_clean["asin"]==asin]
      if len(sub_category) > 0:
        sub_category = sub_category["sub_category"].iloc[0]
        tmp[index[sub_category]] += 1
    output.append(tmp)
    current = current + biastime
  return output

output = historyCounts(datetime(2017,1,1), datetime(2018,10,1), relativedelta(months=1))

'''
output =
[[2, 0, 8496, 0, 10, 7, 0, 97, 5, 0, 0],
 [0, 0, 6453, 0, 6, 3, 0, 75, 0, 0, 0],
 [0, 0, 7401, 2, 6, 5, 0, 44, 0, 0, 1],
 [1, 0, 5979, 0, 6, 3, 0, 54, 1, 0, 1],
 [1, 0, 5565, 0, 4, 4, 0, 58, 0, 1, 0],
 [0, 1, 5192, 0, 6, 1, 0, 59, 0, 0, 0],
 [1, 0, 5330, 0, 4, 2, 0, 29, 0, 1, 0],
 [0, 0, 5892, 1, 3, 0, 0, 46, 0, 0, 1],
 [0, 0, 5865, 1, 5, 0, 1, 44, 0, 0, 0],
 [0, 0, 4587, 0, 2, 1, 0, 50, 0, 1, 0],
 [1, 0, 4142, 0, 8, 0, 0, 34, 0, 0, 1],
 [1, 0, 3893, 0, 2, 0, 0, 68, 1, 0, 0],
 [0, 0, 4766, 0, 3, 0, 0, 66, 0, 0, 0],
 [0, 0, 4389, 0, 4, 5, 0, 45, 0, 1, 2],
 [0, 0, 4869, 0, 5, 12, 0, 49, 0, 0, 0],
 [1, 0, 4542, 0, 9, 12, 0, 49, 0, 1, 0],
 [0, 0, 3749, 1, 5, 5, 0, 29, 0, 0, 1],
 [0, 0, 2936, 0, 7, 6, 0, 25, 1, 1, 0],
 [1, 0, 2347, 0, 3, 12, 0, 22, 1, 1, 0],
 [0, 0, 1644, 0, 6, 1, 0, 11, 0, 0, 0],
 [2, 0, 584, 0, 1, 1, 0, 2, 0, 0, 0]]
'''

2017-01-01 -> 2017-02-01


KeyboardInterrupt: ignored

In [ ]:
def historyTimestep(start, end, biastime):
  timestep = []
  current = start
  while current < end:
    currentTime = current.strftime("%Y-%m-%d")
    timestep.append(currentTime)
    current = current + biastime
  return timestep

sub_category_Counts = pd.DataFrame(output, columns=np.unique(metadata_clean["sub_category"]))
sub_category_Counts.index = historyTimestep(datetime(2017,1,1), datetime(2018,10,1), relativedelta(months=1))
sub_category_Counts

In [ ]:
for column in sub_category_Counts.columns:
  print("#############===")
  print(column)
  sub_category_Counts[column].plot()
  plt.show()
  print()

## 資料切分

In [46]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 標題（Title）TF-IDF分析

In [15]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.cluster import KElbowVisualizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def get_tokens(text):
  lowers = text.lower()
  #remove the punctuation using the character deletion step of translate
  remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
  no_punctuation = lowers.translate(remove_punctuation_map)
  tokens = nltk.word_tokenize(no_punctuation)
  return tokens

def stem_tokens(tokens, stemmer):
  stemmed = []
  for item in tokens:
    stemmed.append(stemmer.stem(item))
  return stemmed

def filterTXTnoisy(txt):
  tokens = get_tokens(txt)
  filtered = [w for w in tokens if not w in stopwords.words('english')]
  stemmer = PorterStemmer()
  return " ".join(stem_tokens(filtered, stemmer))

In [17]:
def extractTXT(x):
  txt = f'{x["title"]}, {" ".join(x["description"])}'
  txt = txt.replace(r"\&amp;","&")
  return filterTXTnoisy(txt)

metadata_clean['text_data'] = metadata_clean.agg(extractTXT, axis=1)
# 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
metadata_clean.to_csv('/content/wordFilter_60group.csv', index=False)

In [20]:
text = metadata_clean['text_data'].tolist()
vectorizor = TfidfVectorizer(min_df=200)
for idx, value in enumerate(text):
    if not isinstance(value, str):
        text[idx] = ''
feature = vectorizor.fit_transform(text).toarray()
len(vectorizor.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


739

In [107]:
n_clusters=100
model = AgglomerativeClustering(n_clusters=n_clusters)
result = model.fit_predict(feature)

In [108]:
metadata_clean['specify'] = pd.Series(result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [109]:
metadata_clean['specify'].value_counts()

4.000     6097
6.000     4083
8.000     1192
56.000    1003
0.000      672
          ... 
97.000      70
88.000      63
86.000      57
62.000      57
91.000      40
Name: specify, Length: 100, dtype: int64

In [110]:
def transform(data):
  res = dict()
  for item, row in data.iterrows():
    res[row['asin']] = row['specify']
  return res
cluster_dict = transform(metadata_clean)

In [111]:
def construct(data):
  if cluster_dict.get(data):
    return cluster_dict[data]
  else:
    return n_clusters+2
ratings['cluster'] = ratings['asin'].apply(construct)

In [112]:
ratings['cluster'].value_counts()

4.000      71933
6.000      58478
8.000      16744
50.000     15553
102.000    11037
           ...  
31.000       385
60.000       289
86.000       265
90.000       241
91.000       212
Name: cluster, Length: 100, dtype: int64

In [113]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
  '''
  * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
  * ratings_by_user: dict 利用訓練資料學習的推薦商品
  * method: str
  * score: float
  '''
  total = 0
  for d in ratings_testings_by_user:
      if d in ratings_by_user:
          total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

  score = total / len(ratings_testings)
  return score

def recommender(training_data, group, time, users=[], k=10):
  if group > n_clusters-1:
    group = n_clusters+2
  target = training_data[(training_data['cluster'] == group)&
                (training_data['DATE']>=time)&
                (training_data['overall']>=training_data['overall'].mean())]
  brand_2 = target['asin'].value_counts().sort_values(ascending=False).keys()[:k]
  recommendations = {user: brand_2.tolist() for user in users}
  return recommendations

def create_data(start, end):
  ratings_trainings = ratings[ 
      (ratings['DATE'] <  start)
  ]
  ratings_testings = ratings[
      (ratings['DATE'] >= start) & 
      (ratings['DATE'] <= end)
  ]
  ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
  ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
  users = list(ratings_testings_by_user.keys())
  return ratings_trainings, ratings_testings, ratings_testings_by_user, users

In [114]:
# time_list = ['2018-07-01','2018-05-01', '2018-01-01', '2016-01-01']
time_period_list = [1, 2, 4, 6, 12]
verify_periods = [['2018-09-01', '2018-09-30'],
          ['2018-08-01', '2018-08-30'],
          ['2017-09-01', '2017-09-30']]

score_list = []
index = []
for start, end in verify_periods:
  y,m,d = np.int32(start.split("-"))
  for period in time_period_list:
    trainStart = (datetime(y,m,d)-relativedelta(months=period)).strftime("%Y-%m-%d")
    print([trainStart, start, end])
    index.append("/".join([start, end, str(period)]))
    ratings_trainings, ratings_testings, ratings_testings_by_user, users = create_data(start, end)
    tmp = list()
    for i in range(n_clusters+1):
      ratings_by_user = recommender(ratings_trainings, i, trainStart, users)
      score = evaluate(ratings_testings_by_user, ratings_by_user)
      tmp.append(score)
    score_list.append(tmp)


['2018-08-01', '2018-09-01', '2018-09-30']
['2018-07-01', '2018-09-01', '2018-09-30']
['2018-05-01', '2018-09-01', '2018-09-30']
['2018-03-01', '2018-09-01', '2018-09-30']
['2017-09-01', '2018-09-01', '2018-09-30']
['2018-07-01', '2018-08-01', '2018-08-30']
['2018-06-01', '2018-08-01', '2018-08-30']
['2018-04-01', '2018-08-01', '2018-08-30']
['2018-02-01', '2018-08-01', '2018-08-30']
['2017-08-01', '2018-08-01', '2018-08-30']
['2017-08-01', '2017-09-01', '2017-09-30']
['2017-07-01', '2017-09-01', '2017-09-30']
['2017-05-01', '2017-09-01', '2017-09-30']
['2017-03-01', '2017-09-01', '2017-09-30']
['2016-09-01', '2017-09-01', '2017-09-30']


In [115]:
pd.options.display.float_format = '{:,.3f}'.format
table = pd.DataFrame(score_list, index=index)
table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
2018-09-01/2018-09-30/1,0.000,0.005,0.000,0.000,0.124,0.002,0.020,0.000,0.019,0.000,0.005,0.000,0.002,0.002,0.010,0.024,0.002,0.002,0.000,0.010,0.005,0.002,0.002,0.002,0.005,0.000,0.008,0.002,0.019,0.000,0.003,0.000,0.000,0.012,0.008,0.002,0.000,0.007,0.002,0.000,...,0.000,0.010,0.000,0.000,0.002,0.002,0.014,0.003,0.000,0.017,0.000,0.000,0.000,0.000,0.002,0.012,0.002,0.003,0.000,0.000,0.008,0.000,0.000,0.005,0.003,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.002,0.000,0.010,0.007,0.000,0.002,0.002,0.029
2018-09-01/2018-09-30/2,0.000,0.005,0.002,0.000,0.125,0.002,0.020,0.000,0.017,0.002,0.005,0.000,0.002,0.003,0.014,0.024,0.002,0.005,0.003,0.010,0.005,0.002,0.002,0.002,0.008,0.000,0.008,0.002,0.020,0.000,0.005,0.000,0.002,0.012,0.008,0.002,0.000,0.005,0.002,0.002,...,0.000,0.010,0.000,0.000,0.002,0.002,0.014,0.003,0.000,0.019,0.000,0.000,0.002,0.000,0.003,0.019,0.003,0.003,0.002,0.000,0.008,0.000,0.000,0.005,0.003,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.003,0.000,0.010,0.007,0.000,0.002,0.002,0.032
2018-09-01/2018-09-30/4,0.000,0.005,0.002,0.000,0.122,0.002,0.007,0.000,0.015,0.003,0.005,0.000,0.002,0.002,0.014,0.024,0.002,0.002,0.007,0.008,0.007,0.002,0.000,0.002,0.010,0.000,0.008,0.002,0.020,0.000,0.005,0.000,0.002,0.012,0.008,0.002,0.000,0.005,0.002,0.002,...,0.000,0.010,0.000,0.000,0.002,0.002,0.014,0.003,0.000,0.019,0.000,0.000,0.003,0.000,0.003,0.017,0.003,0.003,0.002,0.000,0.014,0.000,0.000,0.005,0.003,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.003,0.000,0.010,0.007,0.000,0.002,0.002,0.031
2018-09-01/2018-09-30/6,0.000,0.005,0.002,0.000,0.097,0.003,0.000,0.000,0.015,0.002,0.005,0.000,0.002,0.002,0.014,0.024,0.002,0.002,0.007,0.008,0.007,0.002,0.002,0.002,0.010,0.000,0.008,0.003,0.020,0.000,0.003,0.000,0.002,0.012,0.008,0.002,0.000,0.005,0.002,0.002,...,0.000,0.010,0.000,0.000,0.002,0.002,0.014,0.003,0.000,0.019,0.002,0.000,0.003,0.000,0.003,0.017,0.003,0.003,0.000,0.000,0.014,0.000,0.003,0.005,0.002,0.000,0.003,0.005,0.000,0.000,0.000,0.000,0.003,0.000,0.008,0.007,0.000,0.002,0.002,0.031
2018-09-01/2018-09-30/12,0.000,0.012,0.002,0.000,0.090,0.002,0.002,0.002,0.015,0.003,0.005,0.000,0.002,0.002,0.014,0.017,0.002,0.002,0.007,0.010,0.003,0.000,0.000,0.007,0.014,0.000,0.008,0.003,0.019,0.000,0.003,0.000,0.000,0.012,0.008,0.002,0.000,0.005,0.000,0.002,...,0.000,0.010,0.000,0.000,0.002,0.002,0.014,0.002,0.000,0.019,0.000,0.000,0.002,0.000,0.003,0.019,0.003,0.003,0.000,0.000,0.012,0.000,0.000,0.005,0.002,0.000,0.003,0.005,0.000,0.000,0.000,0.000,0.002,0.000,0.008,0.007,0.000,0.002,0.002,0.029
2018-08-01/2018-08-30/1,0.000,0.002,0.001,0.002,0.105,0.007,0.017,0.003,0.020,0.003,0.007,0.000,0.001,0.011,0.004,0.011,0.008,0.004,0.004,0.007,0.005,0.001,0.002,0.002,0.006,0.002,0.004,0.000,0.001,0.001,0.007,0.000,0.000,0.015,0.006,0.001,0.000,0.004,0.001,0.001,...,0.004,0.006,0.014,0.000,0.001,0.001,0.007,0.002,0.000,0.008,0.004,0.002,0.000,0.002,0.002,0.006,0.000,0.000,0.006,0.000,0.004,0.001,0.000,0.002,0.002,0.000,0.032,0.002,0.004,0.000,0.000,0.000,0.001,0.000,0.002,0.004,0.000,0.001,0.006,0.010
2018-08-01/2018-08-30/2,0.000,0.002,0.001,0.003,0.102,0.007,0.015,0.003,0.020,0.003,0.007,0.000,0.001,0.011,0.004,0.014,0.009,0.005,0.003,0.007,0.004,0.001,0.002,0.002,0.006,0.002,0.004,0.001,0.006,0.002,0.007,0.000,0.000,0.015,0.006,0.001,0.001,0.007,0.001,0.001,...,0.004,0.008,0.014,0.000,0.002,0.002,0.007,0.002,0.001,0.008,0.004,0.003,0.000,0.003,0.002,0.007,0.001,0.003,0.006,0.000,0.004,0.001,0.000,0.002,0.002,0.000,0.033,0.002,0.009,0.000,0.000,0.001,0.001,0.000,0.002,0.004,0.000,0.001,0.006,0.010
2018-08-01/2018-08-30/4,0.000,0.002,0.002,0.003,0.082,0.009,0.010,0.004,0.020,0.003,0.007,0.001,0.001,0.011,0.004,0.013,0.009,0.004,0.004,0.007,0.004,0.002,0.002,0.002,0.006,0.003,0.004,0.002,0.007,0.002,0.0

### AC課程TF IDF

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 計算商品用標題所表示的 tfidf 矩陣
df = metadata_clean.drop_duplicates('title')
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['title'])

In [65]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

In [ ]:
mapping = pd.Series(df.index,index = df['title'])
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

In [ ]:
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [ ]:
recommend_items(items=["Loud 'N Clear&trade; Personal Sound Amplifier"], k=10)

['6546546450',
 'B002W3172S',
 'B001FT5DMU',
 'B008L2UR66',
 'B00WUUW18W',
 'B00QUAL0BW',
 'B01G689TOC',
 'B00DG53QSE',
 'B00632SBHS',
 'B00C69IXIY']

In [ ]:
for asin in recommend_items(items=["Loud 'N Clear&trade; Personal Sound Amplifier"], k=10):
  print(asin," : ",metadata[metadata["asin"]==asin]["title"].values)

6546546450  :  ["Loud 'N Clear&trade; Personal Sound Amplifier"]
B002W3172S  :  ['Super Loud Air Horn (2 oz)']
B001FT5DMU  :  ['Juice Organics Lip Amplifier - 0.5 oz.']
B008L2UR66  :  ['Dove Mousse Nourishing Amplifier 7 (Pack of 3)']
B00WUUW18W  :  ["Bare Escentuals bareMinerals 'Loud &amp; Clear' Lip Sheer (1.9 g) - Unboxed - Magenta Vibe"]
B00QUAL0BW  :  ['bareMinerals Loud &amp; Clear Lip Duo - Amped Pink/Tangerine Trance']
B01G689TOC  :  ['Bare Minerals Loud and Clear Lip Sheer Pink Pulse 0.07oz in retail box']
B00DG53QSE  :  ['In Line Amplifier Tone Control 40Db']
B00632SBHS  :  ['Prive Shining Weightless Amplifier 6.8 oz (Pack of 2)']
B00C69IXIY  :  ['Safe and Sound Lockable Safety Pill Cutter']


## 產生推薦

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{}

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.0